In [1]:
# // import tags from './tags.json';
import json
import pandas as pd
import time
import requests
import numpy as np

In [10]:
# %pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
# load from csv
df = pd.read_csv('favs.csv')
df.drop(columns='Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1'.split(','), inplace=True)
df = df.astype({'id': 'int64'})
df.to_csv('favs_cleaned.csv', index=False)

In [24]:
df.to_feather('favs_cleaned.feather')

In [ ]:
# @title Feather
limit = 1000
# get url for user and page


def get_url(user, page):
    return f"https://gelbooru.com/index.php?page=dapi&q=index&json=1&limit={limit}&" + \
        f"s=post&tags=fav:{user}&pid={page}&" + \
        "api_key=c3ca8d6069fba65a4fd6e81e412712c6607215b3f4710e62f6ee3de44a8350df&user_id=347815"


file_name = "favs.feather"
# load df from csv
# favs_df = pd.read_csv("favs.csv")
favs_df: pd.DataFrame = pd.read_feather(file_name)
favs_df.replace({'True': True, 'False': False,
                'true': True, 'false': False}, inplace=True)

# favs_df.reset_index(inplace=True, drop=False)
display(favs_df.tail())
# favs_df.to_feather('_'+file_name)
# print(favs_df.shape)
# return
# favs_df.set_index(['id', 'user_id'], inplace=True)
# favs_df.to_feather(file_name)

# max_id = favs_df.index.get_level_values('user_id').max()
max_id = favs_df['user_id'].max()
id_to_start = max_id if max_id >= 1 else 1
for user_id in range(id_to_start, 1_000_000):
    print(f"User: {user_id}")
    page = 0
    user_favs = pd.DataFrame()
    print('Page: ', end="")
    while True:
        print(f"{page}; ", end="")
        url = get_url(user_id, page)
        response = requests.request("GET", url)
        try:
            response = response.json()
        except json.decoder.JSONDecodeError as e:
            print(response.text)
            break
        if 'post' not in response:
            break
        user_favs = user_favs.append(pd.DataFrame(response['post']))
        count = response['@attributes']['count']
        page += 1
        if page * limit > count:
            break
    print('')
    # if len(user_favs) == 0:
    #     continue
    user_favs['user_id'] = user_id
    save_start_time = time.time_ns() // 1000000
    try:
        user_favs.replace({'True': True, 'False': False,
                         'true': True, 'false': False}, inplace=True)
        favs_df = favs_df.append(user_favs, ignore_index=True)
        appended = True
        if user_id % 100 == 0:
            favs_df.to_feather(file_name)
            favs_df.to_feather(f"{str(save_start_time)}_{file_name}")
            save_duration = time.time_ns() // 1000000 - save_start_time
            print(f'Saved in {save_duration} ms')
            print(f"Saved {favs_df.shape}")
    except KeyboardInterrupt:
        print('Interrupted')
        if not appended:
            favs_df = favs_df.append(user_favs, ignore_index=True)
        appended = False
        favs_df.to_feather(file_name)
        raise KeyboardInterrupt


In [2]:
import sqlite3

connection = sqlite3.connect('booru.db')

# favs_df['user_id'].drop_duplicates().to_sql(
#     'users', connection,
#     if_exists='replace',
#     dtype={'user_id': 'INTEGER'},
#     index=False,
#     method='multi'
# )


# favs_df.drop('user_id', axis=1).drop_duplicates().to_sql(
#     'posts',
#     connection,
#     if_exists='replace',
#     dtype={'id': 'INTEGER'},
#     index=False,
#     method='multi',
#     chunksize=1000
# )

# favs_df[['user_id', 'id']] \
#     .rename(columns={"id": "post_id"}) \
#     .drop_duplicates() \
#     .to_sql(
#         'favs',
#         connection,
#         if_exists='append',
#         dtype={'user_id': 'INTEGER', 'post_id': 'INTEGER'},
#         index=False,
#         method='multi',
#         chunksize=1000,
# )
max_id = connection.execute("SELECT MAX(user_id) FROM users").fetchone()[0]
max_id

80325

In [57]:
connection.close()

In [10]:
limit = 1000
# get url for user and page


def get_url(user, page):
    return f"https://gelbooru.com/index.php?page=dapi&q=index&json=1&limit={limit}&" + \
        f"s=post&tags=fav:{user}&pid={page}&" + \
        "api_key=c3ca8d6069fba65a4fd6e81e412712c6607215b3f4710e62f6ee3de44a8350df&user_id=347815"


max_id = connection.execute("SELECT MAX(user_id) FROM users").fetchone()[0]
id_to_start = max_id if max_id >= 1 else 1
for user_id in range(id_to_start, 1_000_000):
    print(f"User: {user_id}")
    page = 0
    user_favs = pd.DataFrame()
    print('Page: ', end="")
    while True:
        print(f"{page}; ", end="")
        url = get_url(user_id, page)
        response = requests.request("GET", url)
        try:
            response = response.json()
        except json.decoder.JSONDecodeError as e:
            print(response.text)
            break
        if 'post' not in response:
            break
        user_favs = user_favs.append(pd.DataFrame(response['post']))
        count = response['@attributes']['count']
        page += 1
        if page * limit > count:
            break
    print('')
    if len(user_favs) == 0:
        continue

    connection.execute(
        "INSERT OR IGNORE INTO users (user_id) VALUES (?)", (user_id,))

    save_start_time = time.time_ns() // 1000000
    user_favs.replace({'True': True, 'False': False,
                       'true': True, 'false': False}, inplace=True)
    connection.executemany(
        "INSERT OR IGNORE INTO posts (id, created_at, score, width, height, md5, directory, image, rating, source, change, owner, creator_id, parent_id, sample, preview_height, preview_width, tags, title, has_notes, has_comments, file_url, preview_url, sample_url, sample_height, sample_width, status, post_locked, has_children) " +
        "VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
        user_favs.values.tolist()
    )

    user_favs['user_id'] = user_id
    connection.executemany(
        "INSERT OR IGNORE INTO favs (user_id, post_id) VALUES (?, ?)",
        user_favs[['user_id', 'id']].rename(
            columns={"id": "post_id"}).values.tolist()
    )
    connection.commit()

    save_duration = time.time_ns() // 1000000 - save_start_time
    print(f'Saved in {save_duration} ms')


User: 153023
Page: 0; 
Saved in 23 ms
User: 153024
Page: 0; 
User: 153025
Page: 0; 
User: 153026
Page: 0; 
User: 153027
Page: 0; 
User: 153028
Page: 0; 
User: 153029
Page: 0; 
Saved in 98 ms
User: 153030
Page: 0; 
User: 153031
Page: 0; 
User: 153032
Page: 0; 
Saved in 22 ms
User: 153033
Page: 0; 
Saved in 72 ms
User: 153034
Page: 0; 
User: 153035
Page: 0; 
Saved in 5 ms
User: 153036
Page: 0; 
User: 153037
Page: 0; 
User: 153038
Page: 0; 
User: 153039
Page: 0; 
User: 153040
Page: 0; 
User: 153041
Page: 0; 
Saved in 11 ms
User: 153042
Page: 0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 
Saved in 1411 ms
User: 153043
Page: 0; 
User: 153044
Page: 0; 
Saved in 8 ms
User: 153045
Page: 0; 
User: 153046
Page: 0; 
Saved in 19 ms
User: 153047
Page: 0; 
Saved in 84 ms
User: 153048
Page: 0; 
User: 153049
Page: 0; 
User: 153050
Page: 0; 
Saved in 231 ms
User: 153051
Page: 0; 
Saved in 8 ms
User: 153052
Page: 0; 
Saved in 74 ms
User: 153053
Page: 0; 
User: 153054
Page: 0; 
User: 153055
Page: 0; 
Saved in 6 ms
User:

In [33]:
got_cols = ['id', 'created_at', 'score', 'width', 'height', 'md5', 'directory',
            'image', 'rating', 'source', 'change', 'owner', 'creator_id',
            'parent_id', 'sample', 'preview_height', 'preview_width', 'tags',
            'title', 'has_notes', 'has_comments', 'file_url', 'preview_url',
            'sample_url', 'sample_height', 'sample_width', 'status', 'post_locked',
            'has_children']
print(len(got_cols))

raw_cols = "id,created_at,score,width,height,md5,directory,image,rating,source,change,owner,creator_id,parent_id,sample,preview_height,preview_width,tags,title,has_notes,has_comments,file_url,preview_url,sample_url,sample_height,sample_width,status,post_locked,has_children,user_id"
print(len(raw_cols.split(',')))


29
30


In [100]:
# favs_df.set_index(['id', 'user_id'])

# favs_df['has_notes'] = favs_df['has_notes'].astype(bool)
# favs_df.dtypes
favs_df.dtypes.to_list()
# favs_df.reset_index().to_feather('favs.feather')

[dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('bool'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('bool'),
 dtype('int64')]

In [22]:

with open('images.json') as f:
    tags = json.load(f)['post']
    data = pd.json_normalize(tags)
    data['tags'] = data['tags'].apply(lambda x: x.split(' '))
    # print(data)
    display(data['tags'])
    data.to_csv('images.csv', index=False)


0      [2girls, 4boys, :3, aoki_(fumomo), aqua_footwe...
1      [abs, absurdly_long_hair, astolfo_(fate), asto...
2      [1girl, absurdres, after_anal, after_vaginal, ...
3      [1girl, absurdres, after_anal, aftersex, all_f...
4      [1girl, absurdres, after_vaginal, aftersex, al...
                             ...                        
995    [1girl, :d, absurdres, bangs, black_hair, blun...
996    [3girls, african_wild_dog_(kemono_friends), af...
997    [1girl, absurdres, anya_(spy_x_family), black_...
998    [1girl, blonde_hair, blue_legwear, blue_sky, b...
999    [blue_sky, blue_theme, building, commentary_re...
Name: tags, Length: 1000, dtype: object

In [12]:

with open('tags.json') as f:
    tags = json.load(f)['tag']
    data = pd.json_normalize(tags)
    print(data)
    data.to_csv('tags.csv', index=False)

          id         name    count  type  ambiguous
0     152532        1girl  4699504     0          0
1    1053536  sole_female  3957666     0          1
2      12336         solo  3598649     0          0
3        262      highres  3300358     5          0
4        265    long_hair  3072974     0          0
..       ...          ...      ...   ...        ...
995     1633       incest    27800     0          0
996   167746  dragon_ball    27701     3          0
997   132813  waist_apron    27700     0          0
998   365621     footwear    27640     0          0
999    45162        :&lt;    27632     0          0

[1000 rows x 5 columns]


In [ ]:
# read csv into dataframe
tags_df = pd.read_csv('tags.csv')
images_df = pd.read_csv('images.csv')

